<a href="https://colab.research.google.com/github/kelecheeii/Loan-Status-Prediction/blob/main/ML_Final_Project_Group_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kelechi V. Mbibi

30221021

Machine Learning (ENEL 682) Project

April 2024

**About Dataset**

The loan approval dataset is a collection of financial records and associated information used to determine the eligibility of individuals or organizations for obtaining loans from a lending institution. It includes various factors such as cibil score, income, employment status, loan term, loan amount, assets value, and loan status. This dataset is commonly used in machine learning and data analysis to develop models and algorithms that predict the likelihood of loan approval based on the given features.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files

upload_d = files.upload()

Saving loan_approval_dataset.xlsx to loan_approval_dataset.xlsx


In [ ]:
# read the data.csv file into our workbook
data = pd.read_csv ('/content/loan_approval_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/loan_approval_dataset.csv'

In [ ]:
# next, we print the info of our columns
data.info()

In [ ]:
#there is a space before each feature

In [ ]:
# we go on to separate the dataset into features (X) and target (y) whilst dropping the 'loan_id' column because it is inconsiquential to our analysis
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

In [ ]:
X

In [ ]:
y

In [ ]:
# checking for missing values in X and y
print("missing values in X: \n{}".format(X.isnull().sum()))
print("\n")
print("missing values in y: {}".format(y.isnull().sum()))

In [ ]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {} ".format(y.shape))

In [ ]:
#Visualizing the number of approvals and rejections
colors = ['green', 'red']

sns.countplot(x = data[' loan_status'], palette=colors)

for p in plt.gca().patches:
    plt.text(p.get_x() + p.get_width() / 2.,
             p.get_height(),
             '{}'.format(p.get_height()),
             ha='center',
             va='bottom')

plt.show()

In [ ]:
#segregating the loan applicants according to income level
def income_level(value):
    if 100000 <= value <= 1000000:
        return "Low"
    elif 1000001 <= value <= 4000000:
        return "Low-Middle"
    elif 4000001 <= value <= 7500000:
        return "Upper-Middle"
    elif 7500001 <= value <= 10000000:
        return "High"
    else:
        return "Very High Income"

data['Income level'] = data[' income_annum'].apply(income_level)

In [ ]:
income = data['Income level'].value_counts().reset_index()
income = income.rename(columns={'Income level':'Income Levels', 'count': "Number of Applicants"})
income

In [ ]:
# Ploting a pie chart of loan applicants according to income level

plt.pie(income['Number of Applicants'], labels=income['Income Levels'], autopct="%1.1f%%")
plt.title("Number of Applicants in Each Income Level")
plt.show()

In [ ]:
# Here, we encode categorical values values, scale numeric values and puts it into a pre-processing pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

numeric_features = ['income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'no_of_dependents',
                    'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['education', 'self_employed']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer,  make_column_selector(dtype_exclude="object")),
        ('cat', categorical_transformer,  make_column_selector(dtype_include="object"))])

In [ ]:
# we instatiate the pipeline with Logistic regression as the placeholder
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', LogisticRegression(max_iter=1000))])

In [ ]:
# split our data into 70% training and 30% testing splits
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=419)

In [ ]:
# using gridsearch, we find the best model with the best parameters that gives us the best results
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'model': [LogisticRegression(max_iter=1000)],
     'model__C': [0.1, 1, 10, 100],
     'preprocessor': [preprocessor]},


    {'model': [KNeighborsClassifier()],
     'preprocessor': [preprocessor],
     'model__weights': ['uniform', 'distance'],
     'model__n_neighbors': ['1','3','5','7','9','11']},    # number of neighbors

    {'model': [GradientBoostingClassifier(random_state=0)],
     'preprocessor': [preprocessor],
     'model__n_estimators': [100, 200, 300, 400, 500],    # number of trees in the forest
     'model__max_depth': [1],         # maximun depth of the trees
     'model__learning_rate': [0.01, 0.1, 1, 10, 100]},

    {'model': [SVC()],
     'preprocessor': [preprocessor],
     'model__C': [0.1, 1, 10, 100],
     'model__kernel': ['linear','rbf', 'poly']}] #types of svc



grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid_search.cv_results_['mean_train_score'][grid_search.best_index_]))
print("Best cross-validation test score: {:.2f}".format(grid_search.best_score_))
print("Test-set score: {:.2f}".format(grid_search.score(X_test, y_test)))

In [ ]:
# Define the variables
best_params = grid_search.best_params_
best_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]
best_cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)

# Create the DataFrame
best_results = pd.DataFrame({
    'Metric': ['Best cross-validation train Score', 'Best cross-Validation test Score', 'Test-set score'],
    'Value': [best_train_score, best_cv_score, test_score]
})

# Display DataFrame
best_results

After comparing four classification models on our dataset; the results suggest that a Gradient Boosting Classifier model with specific hyperparameters achieved impressive performance on both the training and testing datasets:

Best Parameters:

The best model selected is a GradientBoostingClassifier with the following hyperparameters:
max_depth: 1
n_estimators: 500
learning_rate: 0.1
The preprocessing pipeline includes standard scaling for numerical features and one-hot encoding for categorical features.

Performance Metrics:

Best Cross-Validation Train Score: 0.99
This indicates that the model achieved a high level of accuracy (99%) on the training data during cross-validation. It suggests that the model effectively learned the patterns in the training data.
Best Cross-Validation Test Score: 0.97
The model also performed exceptionally well on unseen data, achieving a cross-validation accuracy of 97%. This suggests that the model has good generalization capability and is not overfitting to the training data.
Test-Set Score: 0.98
The accuracy of 98% on the test set further validates the model's performance. It indicates that the model maintains high accuracy when applied to completely unseen data, reinforcing confidence in its effectiveness.

Insights:

The high scores across all metrics suggest that the selected model with the specified hyperparameters and preprocessing steps is well-suited for the classification task.
The model demonstrates strong predictive power, achieving near-perfect accuracy on both the training and testing datasets.
The preprocessing steps, including standard scaling and one-hot encoding, effectively prepared the data for modeling, contributing to the model's overall performance.

Overall, the results indicate that the Gradient Boosting Classifier, with appropriate tuning and preprocessing, can effectively classify the data and make accurate predictions.

In [ ]:
#now we do error analysis to conclude using a confusion matrix heatmap
from sklearn.metrics import accuracy_score
y_test_pred = best_model.predict(X_test)
validation_score = accuracy_score(y_test_pred, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
Conf_Mat = confusion_matrix(y_test,y_test_pred )
my_labels = ['Rejected','Approved']
print(Conf_Mat)

In [ ]:
sns.heatmap(Conf_Mat, square=True, annot=True, cbar=False, fmt='d', xticklabels=my_labels, yticklabels=my_labels)
plt.xlabel('Predicted Value')
plt.ylabel('True Value')
plt.title('Confusion Matrix')

we conducted an error analysis for our model's predictions and got the following insights;

Accuracy is a measure of how often the classifier is correct.

It is calculated as (TP + TN) / (TP + TN + FP + FN).
In this case, the accuracy can be calculated as (780 + 475) / (780 + 475 + 17 + 9) = 0.979, or 97.9%.

The high number of true positives (475) and true negatives (780) indicates that the model is making accurate predictions for both classes.

The low number of false positives (17) and false negatives (9) suggests that the model has a low rate of misclassification.

The overall accuracy of 97.9% suggests that the model is performing well in terms of correctly classifying instances from the test dataset.



In [ ]:
grid_search.best_params
